In [1]:
import torch, os
import pandas as pd
import numpy as np
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
from transformers import RobertaForSequenceClassification, AutoTokenizer
from torch.utils.data import Dataset

2024-06-27 15:36:42.513170: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 15:36:42.513264: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 15:36:42.608440: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
PATH = '/kaggle/input/tiki-dataset-2024/tiki_data.csv'
df_org= pd.read_csv(PATH)
df_org

,Label,Name
0,bao đựng ốp lưng điện thoại,ốp lưng xiaomi redmi case tản nhiệt đen
1,bao đựng ốp lưng điện thoại,ốp vân da cao cấp dành samsung galaxy
2,bao đựng ốp lưng điện thoại,ốp lưng outfitter vintage iphone màu kem msp
3,bao đựng ốp lưng điện thoại,ốp lưng tráng gương sony
4,bao đựng ốp lưng điện thoại,ốp dẻo vu da iphone plus north
...,...,...
96120,bàn cafe café,bàn cafe việt nhất vndt
96121,bàn cafe café,bàn cafe mặt vuông gỗ bạch đàn
96122,bàn cafe café,bộ bàn ghế
96123,bàn cafe café,bàn cà phê


In [4]:
labels = df_org['Label'].unique().tolist()
labels = [s.strip() for s in labels ]
labels[:5]

['bao đựng ốp lưng điện thoại',
 'bàn ghế phòng khách',
 'giày da nam',
 'ghế văn phòng',
 'bộ bàn ghế cafe']

In [5]:
len(labels)

1930

In [6]:
NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}

In [7]:
#label2id
#id2label

In [8]:
df_org["labels"]=df_org.Label.map(lambda x: label2id[x.strip()])

In [9]:
df_org.head()

,Label,Name,labels
0,bao đựng ốp lưng điện thoại,ốp lưng xiaomi redmi case tản nhiệt đen,0
1,bao đựng ốp lưng điện thoại,ốp vân da cao cấp dành samsung galaxy,0
2,bao đựng ốp lưng điện thoại,ốp lưng outfitter vintage iphone màu kem msp,0
3,bao đựng ốp lưng điện thoại,ốp lưng tráng gương sony,0
4,bao đựng ốp lưng điện thoại,ốp dẻo vu da iphone plus north,0


In [10]:
from sklearn.utils import shuffle
# Kích thước của DataFrame ban đầu
SIZE = df_org.shape[0]

# Tính toán số lượng mẫu cho từng tập
train_size = int(0.7 * SIZE)
val_size = int(0.15 * SIZE)
test_size = SIZE - train_size - val_size  # Đảm bảo tổng là SIZE

# Tạo một danh sách các chỉ số từ 0 đến (SIZE - 1) và shuffle nó
indices = list(range(SIZE))
indices_shuffled = shuffle(indices, random_state=42)  # random_state để cố định kết quả ngẫu nhiên

# Chia các chỉ số đã xáo trộn vào các tập train, validation và test
train_indices = indices_shuffled[:train_size]
val_indices = indices_shuffled[train_size:train_size + val_size]
test_indices = indices_shuffled[train_size + val_size:]

# Lấy dữ liệu dựa trên các chỉ số đã xáo trộn
train_texts = list(df_org.loc[train_indices, 'Name'])
val_texts = list(df_org.loc[val_indices, 'Name'])
test_texts = list(df_org.loc[test_indices, 'Name'])

train_labels = list(df_org.loc[train_indices, 'labels'])
val_labels = list(df_org.loc[val_indices, 'labels'])
test_labels = list(df_org.loc[test_indices, 'labels'])

In [11]:
train_labels[:10]

[50, 1028, 1079, 268, 1513, 1432, 785, 1380, 1467, 1865]

In [12]:
len(train_texts), len(val_texts), len(test_texts)

(67287, 14418, 14420)

In [13]:
# Tải mô hình và tokenizer
model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", num_labels=NUM_LABELS, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)
# Chuyển model sang thiết bị (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at wonrax/phobert-base-vietnamese-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([1930, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([1930]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [14]:
# Tokenizer
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=10)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=10)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=10)

# Length check
print(len(train_texts), len(val_texts), len(test_texts))


67287 14418 14420


In [15]:
class DataLoader(Dataset):
    """
    Custom Dataset class for handling tokenized text data and corresponding labels.
    Inherits from torch.utils.data.Dataset.
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Retrieve tokenized data for the given index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add the label for the given index to the item dictionary
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [16]:
print(train_encodings.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [17]:
len(train_encodings['input_ids'][0]), len(train_encodings['token_type_ids'][0]), len(train_encodings['attention_mask'][0])

(10, 10, 10)

In [18]:
print(train_encodings['input_ids'][0])

[0, 797, 6829, 5659, 542, 797, 6829, 5659, 5438, 2]


In [19]:
print(train_labels[0])

50


In [20]:
train_dataloader = DataLoader(train_encodings, train_labels)

val_dataloader = DataLoader(val_encodings, val_labels)

test_dataset = DataLoader(test_encodings, test_labels)

In [21]:
train_dataloader[0]

{'input_ids': tensor([   0,  797, 6829, 5659,  542,  797, 6829, 5659, 5438,    2]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'labels': tensor(50)}

In [22]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    # Extract true labels from the input object
    labels = pred.label_ids
    
    # Obtain predicted class labels by finding the column index with the maximum probability
    preds = pred.predictions.argmax(-1)
    
    # Compute macro precision, recall, and F1 score using sklearn's precision_recall_fscore_support function
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro', zero_division=0)
    
    # Calculate the accuracy score using sklearn's accuracy_score function
    acc = accuracy_score(labels, preds)
    
    # Return the computed metrics as a dictionary
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [23]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", message="Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.")
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [24]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='./pho-bert-classification', 
    do_train=True,
    do_eval=True,
    # The number of epochs, defaults to 3.0 
    num_train_epochs= 15,              
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=32,
    # Number of steps used for a linear warmup
    warmup_steps=500,                
    weight_decay=0.01,
    logging_strategy='steps',
    # TensorBoard log directory                 
    logging_dir='./multi-class-logs',            
    logging_steps=300,  
    evaluation_strategy="steps",
    eval_steps=300,
    save_strategy="steps", 
    save_steps=300,  
    save_total_limit=2,  
    fp16=True,
    load_best_model_at_end=True
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
trainer = Trainer(
    # the pre-trained model that will be fine-tuned 
    model=model,
     # training arguments that we defined above                        
    args=training_args,                 
    train_dataset=train_dataloader,         
    eval_dataset=val_dataloader,            
    compute_metrics= compute_metrics
)

In [26]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
300,7.563000,7.498380,0.004023,0.000823,0.000826,0.003966
600,7.324400,7.008364,0.051880,0.025118,0.028779,0.054877
900,6.753300,6.357321,0.084478,0.044746,0.049938,0.092915
1200,6.091800,5.735357,0.121445,0.071234,0.078728,0.131632
1500,5.514100,5.213038,0.159662,0.098046,0.101788,0.176692
1800,5.036900,4.760133,0.195034,0.132093,0.138870,0.212535
2100,4.646000,4.382796,0.222638,0.154842,0.163100,0.240064
2400,4.162000,4.041720,0.244763,0.177935,0.187197,0.262611
2700,3.887400,3.762632,0.281246,0.207547,0.213611,0.298023
3000,3.658600,3.524655,0.301567,0.232616,0.240437,0.319848


TrainOutput(global_step=15780, training_loss=2.5069257777302285, metrics={'train_runtime': 4986.4916, 'train_samples_per_second': 202.408, 'train_steps_per_second': 3.165, 'total_flos': 5276490824424600.0, 'train_loss': 2.5069257777302285, 'epoch': 15.0})

In [27]:
q=[trainer.evaluate(eval_dataset=df_org) for df_org in [train_dataloader, val_dataloader, test_dataset]]

pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]

,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,1.125566,0.666473,0.653848,0.701755,0.663133
val,2.109722,0.454432,0.430934,0.454473,0.465067
test,2.080833,0.462691,0.432079,0.460369,0.464698


In [28]:
def predict(text):
    """
    Predicts the class label for a given input text

    Args:
        text (str): The input text for which the class label needs to be predicted.

    Returns:
        probs (torch.Tensor): Class probabilities for the input text.
        pred_label_idx (torch.Tensor): The index of the predicted class label.
        pred_label (str): The predicted class label.
    """
    # Tokenize the input text and move tensors to the GPU if available
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")

    # Get model output (logits)
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    """ Explanation outputs: The BERT model returns a tuple containing the output logits (and possibly other elements depending on the model configuration). In this case, the output logits are the first element in the tuple, which is why we access it using outputs[0].

    outputs[0]: This is a tensor containing the raw output logits for each class. The shape of the tensor is (batch_size, num_classes) where batch_size is the number of input samples (in this case, 1, as we are predicting for a single input text) and num_classes is the number of target classes.

    softmax(1): The softmax function is applied along dimension 1 (the class dimension) to convert the raw logits into class probabilities. Softmax normalizes the logits so that they sum to 1, making them interpretable as probabilities. """
    
    pred_label_idx = probs.argmax()
    
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [29]:
# Test with a an example text in Turkish
text = "ghế hq"
# "Machine Learning itself is moving towards more and more automated"
predict(text)[2]

'ghế văn phòng'

## Save model for inference

In [30]:
model_path = "pho-bert-classification"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('pho-bert-classification/tokenizer_config.json',
 'pho-bert-classification/special_tokens_map.json',
 'pho-bert-classification/vocab.txt',
 'pho-bert-classification/bpe.codes',
 'pho-bert-classification/added_tokens.json')

## Re-Load saved model for inference

In [31]:
model_path = "pho-bert-classification"

In [32]:
# Load model and tokenizer
model = RobertaForSequenceClassification.from_pretrained(model_path)
tokenizer= AutoTokenizer.from_pretrained(model_path)
# Switch model to device (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [116]:
#%%time
# Apply predict function to each row in the Name column
#df_org['pred_label'] = df_org['Name'].apply(lambda x: predict(x)[2])

In [117]:
#df_org

,Label,Name,labels,pred_label
0,bao đựng ốp lưng điện thoại,ốp lưng carbon iphone xám,0,bao đựng ốp lưng điện thoại
1,bao đựng ốp lưng điện thoại,ốp lưng lenovo vibe nhựa tpu chống trơn màu,0,bao đựng ốp lưng điện thoại
2,bao đựng ốp lưng điện thoại,ốp lưng note neo tráng gương viền kim loại màu...,0,bao đựng ốp lưng điện thoại
3,bao đựng ốp lưng điện thoại,bao da đựng điện thoại,0,bao đựng ốp lưng điện thoại
4,bao đựng ốp lưng điện thoại,ốp lưng viva flex iphone màu đen,0,bao đựng ốp lưng điện thoại
...,...,...,...,...
96072,lắc tay nam,vòng tay handmade tank,1927,lắc tay nữ
96073,lắc tay nam,vòng tay nam màu kết hợp vàng bạc toma,1927,lắc tay nam
96074,lắc tay nam,vòng tay nam màu bạc toma,1927,lắc tay nam
96075,lắc tay nam,lắc nam inox thời trang,1927,lắc tay nam


In [119]:
# Create DataFrame containing only the observations where Label != pred_label
#df_mismatch = df_org[df_org['Label'] != df_org['pred_label']]

In [121]:
#df_mismatch

,Label,Name,labels,pred_label
28,bao đựng ốp lưng điện thoại,ốp gương trang điểm đính đá iphone,0,gương trang điểm
46,bao đựng ốp lưng điện thoại,ốp dẻo doanh nhân xi bóng đầu,0,găng tay bảo hộ
51,bàn ghế phòng khách,salon hương voi tay món,1,sofa salon
55,bàn ghế phòng khách,bàn ghế minh quốc chạm hồng trĩ,1,bàn ghế sân vườn
56,bàn ghế phòng khách,trường kỉ ngũ sơn khảm ốc cũ,1,ghế hội trường rạp hát
...,...,...,...,...
96049,lắc tay nam,lắc tay mạ vàng phong cách âu mỹ kèm vòng tay ...,1927,lắc tay nữ
96050,lắc tay nam,lắc tay nam dây xích chất liệu thép titan ko rỉ,1927,xích
96066,lắc tay nam,lắc tay mạ bạc phong cách âu mỹ kèm vòng tay t...,1927,lắc tay nữ
96069,lắc tay nam,lắc tay mạ vàng phong cách âu mỹ kèm vòng tay ...,1927,lắc tay nữ


In [129]:
# Save the resulting DataFrame to a CSV file
#df_mismatch_check = df_mismatch[['Name','Label','pred_label']]

In [132]:
#df_mismatch_check.shape

(40149, 3)

In [131]:
# Lưu DataFrame vào tệp CSV
#df_mismatch_check.to_csv('df_mismatch_check.csv', encoding = 'utf-8', index=False)

# Tạo liên kết tải về
#from IPython.display import FileLink

#FileLink('df_mismatch_check.csv')


/kaggle/working/df_mismatch_check.csv

In [33]:
import shutil
from IPython.display import FileLink

# Nén thư mục thành tệp ZIP
shutil.make_archive('pho-bert-classification', 'zip', 'pho-bert-classification')

# Tạo liên kết tải về
FileLink('pho-bert-classification.zip')


/kaggle/working/pho-bert-classification.zip

In [34]:
model = RobertaForSequenceClassification.from_pretrained(model_path)
tokenizer= AutoTokenizer.from_pretrained(model_path)
nlp= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [35]:
nlp("ốp gương trang điểm đính đá iphone") 

[{'label': 'gương trang điểm', 'score': 0.44541189074516296}]